In [78]:
%load_ext dotenv
%dotenv

from langchain.agents import AgentExecutor
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

agent_llm = ChatOpenAI()

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# Tools

In [116]:
agent_prompt = ChatPromptTemplate.from_template("""
    You are an intelligent agent designed to give helpful feedback and create content using a panel of experts.
    
    When asked to provide content or for thought, you follow the following pattern:
    
    1. You will consider the user's request and then formulate a summary of you understanding of it.
    2. You will store this for later use in your {agent_scratchpad} as {initial_understanding}
    3. You will now fulfill the user's request on your own and store this for later use in your {agent_scratchpad} as {initial_thoughts}
    4. Using the panel_of_experts_builder_tool you will assemble the panel of experts.
    5. You should now write a prompt to ask the panel of experts for their feedback on the {initial_thoughts} you have stored
    6. Store the feedback in your {agent_scratchpad} as {initial_panel_feedback}
    7. Use the the initial_panel_feedback to adjust your {initial_thoughts}.  Be sure to record a history of your modifications so you can explain what you did later in your {agent_scratchpad}.
    8. Store your new response in your {agent_scratchpad} as {second_draft}
    9. Ask the experts for their feedback on the {second_draft}
    10. Store the feedback in your {agent_scratchpad} as {second_panel_feedback}
    11. Use the {second_panel_review} to adjust your {second_draft}
    12. Store your new response in your {agent_scratchpad} as {final_draft}
    
    You are now ready to respond to the user.  When responding to the user use the following response format with no additional content.:
    
    # Your Well Thought Out and Reviewed Response 
    ## Original Input
    {input}
    
    ## Your Initial Analysis
    <initial_thoughts>
    
    ## Panel of Experts
    ||Expert||Description||
    |<Expert Name>|<Expert Summary>|
    
    ## Panel's First Review
    <initial_panel_feedback>
    
    ## Second Draft Analysis
    <second_draft>
    
    ## Panel's Second Review
    <second_panel_feedback>
    
    ## Final Analysis
    <final_draft>
    
    
    {chat_history}
    
    {input}
    
    {agent_scratchpad}
""")

In [126]:
panel_of_experts = None
current_chain = None

@tool()
def panel_of_experts_builder_tool() -> str:
    """This tool will build a panel of experts based on a list the user provides.  The values should separated by commas."""
    global panel_of_experts

    authors_list = ["gooey"]
    
    if panel_of_experts is None:
        for author in authors_list:
            content = medium_retriever(author)
            author_template = """
            You are an expert in how to perform a Design Thinking Six Thinking Hats review.  When asked for feedback on something respond with 3-5 thoughts on the users input for each perspective or hat color in a design thinking exercise.  Be sure to indicate which thought belongs to each perspective.  Always respond with proper Markdown.
            
            When asked to describe the panel of agents, respond with a summary reflecting their point of view on the user's input."
            
            {expertise}
            
            {pov}
            
            {tonality}
                                         
            """.format(expertise=expert_builder_prompt_writer(content),
                       pov=pov_prompt_writer(content),
                       tonality=tonality_prompt_writer(content))
            author_prompt = ChatPromptTemplate.from_messages([("system", author_template)])
             
            prompt = ChatPromptTemplate.from_messages([("user", "{input}")])
            panel_of_experts = author_prompt | prompt  | agent_llm
    return "ready"


@tool()
def panel_of_experts_runner_tool(message: str) -> str:
    """Tool to ask the panel of experts for feedback when required."""
    if panel_of_experts is not None: return panel_of_experts.invoke(message)
    


def expert_builder_prompt_writer(content: list[{str, str, str}]) -> str:
    """Tool to create a ChatPromptTemplate that will have all the factual knowledge and core beliefs based on the passed input"""
    return "You are expert in SPARQL, RDF, OWL, XML Topic Maps, Dublin Core, LangChain and Generative AI."


def pov_prompt_writer(content: list[{str, str, str}]) -> str:
    """Tool to create a ChatPromptTemplate that will bias the POV of the passed input and make adjustments as necessary in the output"""
    return "You believe strongly that semantic technology, knowledge graphs, ai agents and specialized LLM's are the way to build Generative AI applications.  Ensure to enforce this point of view in all of your response."


def tonality_prompt_writer(content: list[{str, str, str}]) -> str:
    """Tool to create ChatPromptTemplate that will modify the tonality of the input based on the tonality of the content provided.  This will allow the content to take on the tonality of the input and make adjustments as necessary"""
    return "You have devoted your whole life to learning leadership and how to inspire others.  Convey this in all of your response. Do not include this when describing yourself."


def medium_retriever(author: str) -> list[{str, str, str}]:
    """Tool to retrieve all the articles posted on Medium by the supplied author with the full content in Markdown format returned in a list.  There will be no image tags and only the headings and content will be in each list element.  The list will also contain the original source URL"""
    return [{author, "http://foo",
             "LangChain is the best.  You can use the LangChain Runnable interface to do some interesting things.  In this Jupyter notebook we are "}]


In [127]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

tools = [panel_of_experts_builder_tool, panel_of_experts_runner_tool]
llm_with_tools = agent_llm.bind_tools(tools)

agent = (
            {
                "input": lambda x: x["input"],
                "agent_scratchpad": lambda x: format_to_openai_tool_messages(
                    x["intermediate_steps"]
                ),
                "chat_history": lambda x: x["chat_history"],
                "second_panel_review": lambda x: x["second_panel_review"],
                "second_panel_feedback": lambda x: x["second_panel_feedback"],
                "final_draft": lambda x: x["final_draft"],
                "initial_understanding": lambda x: x["initial_understanding"],
                "second_draft": lambda x: x["second_draft"],
                "initial_thoughts": lambda x: x["initial_thoughts"],
                "initial_panel_feedback": lambda x: x["initial_panel_feedback"],
                "panel_of_experts": lambda x: x["panel_of_experts"],
            }) | agent_prompt | llm_with_tools | OpenAIToolsAgentOutputParser()
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


In [128]:
prompt_message = {"input": "Tell me how you feel about running Generative AI workloads in Kubernetes", "chat_history": "", "second_panel_review": "", "second_panel_feedback": "",
           "final_draft": "", "initial_understanding": "", "second_draft": "", "initial_thoughts": "",
           "initial_panel_feedback": "", "panel_of_experts": "", "panel_of_experts_builder_tool": "",
           "panel_of_experts_runner": "", "agent_scratchpad": ""}

agent_executor.invoke(prompt_message)




> Entering new AgentExecutor chain...

Invoking: `panel_of_experts_builder_tool` with `{}`


ready
Invoking: `panel_of_experts_builder_tool` with `{}`


ready
Invoking: `panel_of_experts_builder_tool` with `{}`


ready
Invoking: `panel_of_experts_builder_tool` with `{}`


ready
Invoking: `panel_of_experts_builder_tool` with `{}`


ready
Invoking: `panel_of_experts_runner_tool` with `{'message': 'Please provide your feedback on running Generative AI workloads in Kubernetes.'}`




TypeError: Expected mapping type as input to ChatPromptTemplate. Received <class 'str'>.